In [27]:
import pandas as pd
import numpy as np
import joblib
import os
import time

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [28]:
X_test = pd.read_csv('../Data/X_test.csv')
y_test = np.loadtxt("../Data/y_test.csv", delimiter=",")

test_models = ["LGBM", "ETC", "SVM", "GNB", "CATBOOST"]
model_dir = "../Models/Tuning"

minmax = joblib.load("../Models/Tuning/minmax.pkl")
pt = joblib.load("../Models/Tuning/powertransformer.pkl")

In [29]:
X_minmax_scaled = minmax.transform(X_test)

X_pt_scaled = pt.transform(X_test)

In [30]:
results = []

print("\nEvaluating Models...\n")
for model_name in test_models:
    model_path = os.path.join(model_dir, f"MODEL-{model_name}.pkl")
    
    if not os.path.exists(model_path):
        print(f"Model {model_name} not found, skipping...\n")
        continue
    
    print(f"Evaluating {model_name}...")
    model = joblib.load(model_path)
    
    # Start time
    start_time = time.time()
    
    # Perform prediction
    if model_name == "SVM":
        y_pred = model.predict(X_minmax_scaled)
        y_pred_proba = model.predict_proba(X_minmax_scaled)[:, 1]
    elif model_name == "GNB":
        y_pred = model.predict(X_pt_scaled)
        y_pred_proba = model.predict_proba(X_pt_scaled)[:, 1]
    else:
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # End time
    end_time = time.time()
    
    # Compute Metrics
    accuracy = accuracy_score(y_test, y_pred) * 100
    precision = precision_score(y_test, y_pred, zero_division=0) * 100
    recall = recall_score(y_test, y_pred, zero_division=0) * 100
    f1 = f1_score(y_test, y_pred, zero_division=0) * 100
    roc_auc = roc_auc_score(y_test, y_pred_proba) * 100
    computation_time = end_time - start_time
    
    # Store results in a list
    results.append([model_name, computation_time, accuracy, precision, recall, f1, roc_auc])


Evaluating Models...

Evaluating LGBM...
Evaluating ETC...
Evaluating SVM...
Evaluating GNB...
Evaluating CATBOOST...


In [31]:
print("\nFinal Testing Results:")
print("=" * 80)
print(f"{'Model':<10} {'Time (s)':<12} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1 Score':<12} {'ROC AUC':<12}")
print("=" * 80)
for res in results:
    print(f"{res[0]:<10} {res[1]:<12.4f} {res[2]:<12.2f} {res[3]:<12.2f} {res[4]:<12.2f} {res[5]:<12.2f} {res[6]:<12.2f}")
print("=" * 80)

print("\nAll models evaluated successfully!")


Final Testing Results:
Model      Time (s)     Accuracy     Precision    Recall       F1 Score     ROC AUC     
LGBM       0.0040       99.44        99.06        98.36        98.71        99.95       
ETC        0.1100       99.49        99.29        98.36        98.82        99.94       
SVM        0.1063       98.43        95.62        97.19        96.40        98.79       
GNB        0.0010       95.58        85.56        95.78        90.39        98.18       
CATBOOST   0.0030       99.44        99.06        98.36        98.71        99.95       

All models evaluated successfully!
